In [152]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
from pygobnilp.gobnilp import Gobnilp
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
import collections
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [3]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_100.dat")
df
df_test = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat")
df_test

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


### chi-squared test

In [71]:
#0th order chi2 test
def chi2bool(df):
    chi2 = []
    v = list(df)
    my_list = list(itertools.combinations(v,2))
    y_0, y_1 = [x[0] for x in my_list], [x[1] for x in my_list]
    for i,j in zip(y_0, y_1):
        chi = chi_square(X=i, Y=j, Z=[], data=df, significance_level=0.05)
        chi2.append((i,j,chi))
    true_0 = [x for x in chi2 if True in x]
    false_0 = [x for x in chi2 if False in x]
    return true_0, false_0

### Sorting Functions

In [108]:
def create_permutations(my_list):
    p_permutations = []
    for items in my_list:
        new_list_0 = list(itertools.permutations(items[:2],2))[1]
        p_permutations.append(new_list_0)
    return [x[:2] for x in my_list] + p_permutations

In [10]:
#given 2 lists, append them together and remove duplicates
def remove_dupes(list_1, list_2):
    for i in list_2:
        list_1.append(i)
    remove_dupes = list(set(list_1))
    return remove_dupes

### CONDITIONAL MAIN CODE

In [175]:
#1st order CI
def cond_1_generate(df):
    v = list(df)
    p_permutations = list(itertools.permutations(v,3))
    order_0 = create_permutations(chi2bool(df)[0])
    generated = [x for x in p_permutations if x[:2] not in order_0]
    generated_0 = set(tuple(sorted(l[:2])) for l in generated)
    generate_return = [x for x in generated if x[:2] in generated_0]
    return generate_return

In [176]:
def cond_1_test(df):
    chi2_data = []
    phi = cond_1_generate(df)
    for i,j,k in phi:
        chi2 = chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.05)
        chi2_data.append((i,j,k,chi2))
    return chi2_data

In [178]:
%%time
G = cond_1_test(df)

CPU times: total: 27.3 s
Wall time: 27.4 s


In [186]:
G

[('One', 'Two', 'Three', False),
 ('One', 'Two', 'Four', True),
 ('One', 'Two', 'Five', True),
 ('One', 'Two', 'Six', True),
 ('One', 'Two', 'Seven', False),
 ('One', 'Two', 'Eight', False),
 ('One', 'Two', 'Nine', False),
 ('One', 'Two', 'Ten', True),
 ('One', 'Two', 'Eleven', True),
 ('One', 'Two', 'Twelve', False),
 ('One', 'Two', 'Thirteen', False),
 ('One', 'Two', 'Fourteen', False),
 ('One', 'Two', 'Fifteen', False),
 ('One', 'Two', 'Sixteen', True),
 ('One', 'Two', 'Seventeen', False),
 ('One', 'Two', 'Eighteen', False),
 ('One', 'Two', 'Nineteen', False),
 ('One', 'Two', 'Twenty', False),
 ('One', 'Two', 'TwentyOne', False),
 ('One', 'Two', 'TwentyTwo', False),
 ('One', 'Two', 'TwentyThree', False),
 ('One', 'Two', 'TwentyFour', False),
 ('One', 'Two', 'TwentyFive', False),
 ('One', 'Two', 'TwentySix', False),
 ('One', 'Two', 'TwentySeven', False),
 ('One', 'Two', 'TwentyEight', False),
 ('One', 'Two', 'TwentyNine', False),
 ('One', 'Two', 'Thirty', False),
 ('One', 'Two', 'Thi

In [188]:
G_true = [x for x in G if True in x]
len(G_true), G_true

(773,
 [('One', 'Two', 'Four', True),
  ('One', 'Two', 'Five', True),
  ('One', 'Two', 'Six', True),
  ('One', 'Two', 'Ten', True),
  ('One', 'Two', 'Eleven', True),
  ('One', 'Two', 'Sixteen', True),
  ('One', 'Two', 'ThirtyTwo', True),
  ('One', 'Six', 'Two', True),
  ('One', 'Six', 'Four', True),
  ('One', 'Six', 'Five', True),
  ('One', 'Six', 'Eight', True),
  ('One', 'Six', 'Nine', True),
  ('One', 'Six', 'Ten', True),
  ('One', 'Six', 'Eleven', True),
  ('One', 'Six', 'TwentyNine', True),
  ('One', 'Six', 'ThirtyTwo', True),
  ('One', 'Six', 'ThirtyFour', True),
  ('One', 'Sixteen', 'Two', True),
  ('One', 'Sixteen', 'Four', True),
  ('One', 'Sixteen', 'Eight', True),
  ('One', 'Sixteen', 'Ten', True),
  ('One', 'Sixteen', 'ThirtyTwo', True),
  ('One', 'Sixteen', 'ThirtyFour', True),
  ('One', 'ThirtyTwo', 'Two', True),
  ('One', 'ThirtyTwo', 'Four', True),
  ('One', 'ThirtyTwo', 'Six', True),
  ('One', 'ThirtyTwo', 'Eleven', True),
  ('One', 'ThirtyFour', 'Two', True),
  ('One'

In [11]:
#returns TRUE/FALSE Xs and Ys in a tuple of (X,Y)
def conditional_sort(df):
    output_false = []
    output_true = []
    my_list = [x for x in chi2bool(df) if False in x]
    my_list_2 = [x for x in chi2bool(df) if True in x]
    for i,j in zip(my_list, my_list_2):
        output_true.append(j[1])
        output_false.append(i[1])
    return output_true, output_false

In [85]:
conditional_sort(df)

([], [])

In [12]:
#returns permutations of x,y,z where x,y does not repeat e.g (a,b,c), (b,a,c)
def conditional_permute(df):
    a = conditional_sort(df)[1] #obtains the list of false outputs, false output means dependent
    k = list(itertools.permutations(df,3))
    permute_list = []
    for i in a:
        permute = list(itertools.permutations(i))
        permute_list.append(permute[1]) #returns a list of permuted items from a
    for items in permute_list:
        a.append(items) #adds all possible permutations to a 
    my_list = [x for x in k if all(x[:2] != y[:2] for y in a)] #checks if all elements in x[:2] != y[:2]
    my_list_2 = [x[:2] for x in my_list]
    a0 = list(tuple(sorted(l)) for l in my_list_2)
    output = [x for x in my_list if x[:2] in a0] #the three lines fixes all the permutations 
    return output

In [13]:
def conditional_1st_order(df):
    b0 = conditional_permute(df)
    b_100 = []
    for i,j,k in b0:
        chi2 = chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.05)
        b_100.append((i,j,k, chi2))
    b_100_true = [x for x in b_100 if True in x]
    b_100_false = [x for x in b_100 if False in x]
    return b_100_true, b_100_false

In [83]:
#a function to merge and remove dupes and add permutationes to remove etc (its confusing)
def permute_0_1(df):
    
    cond_0 = chi2bool(df)[0]
    true_list_0 = [x[:2] for x in cond_0]
    
    cond_1 = conditional_1st_order(df)
    true_list_1 = [x[:2] for x in cond_1[0]]
    
    true_list = true_list_0 + true_list_1

    chi2_ind = list(set(tuple(sorted(l)) for l in true_list)) #ensures no DUPLICATE permutations in the list
    
    chi2_permute = []
    for i in chi2_ind:
        delta = list(itertools.permutations(i, 2))
        chi2_permute.append(delta[1])
    
    chi2_permutations = chi2_ind + chi2_permute #0th and 1st order + their permutations (singular)
    
#     v = list(df)
#     k = list(itertools.permutations(v, n)) #nPr 
#     t = [x for x in k if x[:2] not in chi2_permutations]
    
#     t_1 = list(set(tuple(sorted(l[:2])) for l in t))
    
#     t_final = [x for x in t if x[:2] not in t_1]
    
    return chi2_permutations

In [84]:
cond_0 = chi2bool(df)[0]
true_list_0 = [x[:2] for x in cond_0]

cond_1 = conditional_1st_order(df)
true_list_1 = [x[:2] for x in cond_1[0]]

true_list = true_list_0 + true_list_1

In [79]:
%%time
k = merger(df)

CPU times: total: 2min 57s
Wall time: 2min 58s


In [82]:
v = list(df)
len(k) / 2

635.0

In [28]:
%%time
remove_list = [x for x in k]
N = 3
n = 3
v = list(df)
while N <= n:
    empty_list = []
    list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
    p_1 = [x[:1] for x in list_permutations]
    p_2 = [x[1:2] for x in list_permutations]
    p = [x[2:] for x in list_permutations]
#     for i,j,k in zip(p_1, p_2, p):
#         chi_square(X=i, Y=j, Z=k, data=df, significance_level=0.95)
#         empty_list.append((i,j,k,chi_square))
#     true_list = [x for x in empty_list if True in x]
#     false_list = [x for x in empty_list if False in x]
    N += 1

CPU times: total: 93.8 ms
Wall time: 87.9 ms


In [36]:
%%time
empty_list = []
N = 3
list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
p_1 = [x[:1] for x in list_permutations]
p_2 = [x[1:2] for x in list_permutations]
p = [x[2:] for x in list_permutations]
# true_list = [x for x in empty_list if True in x]
# false_list = [x for x in empty_list if False in x]

CPU times: total: 31.2 ms
Wall time: 38.3 ms


In [52]:
chi2bool(df),conditional_1st_order(df)

([[0, ('One', 'Two'), False],
  [1, ('One', 'Three'), True],
  [2, ('One', 'Four'), False],
  [3, ('One', 'Five'), False],
  [4, ('One', 'Six'), False],
  [5, ('One', 'Seven'), True],
  [6, ('One', 'Eight'), True],
  [7, ('One', 'Nine'), True],
  [8, ('One', 'Ten'), True],
  [9, ('One', 'Eleven'), False],
  [10, ('One', 'Twelve'), True],
  [11, ('One', 'Thirteen'), True],
  [12, ('One', 'Fourteen'), True],
  [13, ('One', 'Fifteen'), True],
  [14, ('One', 'Sixteen'), False],
  [15, ('One', 'Seventeen'), True],
  [16, ('One', 'Eighteen'), True],
  [17, ('One', 'Nineteen'), True],
  [18, ('One', 'Twenty'), True],
  [19, ('One', 'TwentyOne'), True],
  [20, ('One', 'TwentyTwo'), True],
  [21, ('One', 'TwentyThree'), True],
  [22, ('One', 'TwentyFour'), True],
  [23, ('One', 'TwentyFive'), True],
  [24, ('One', 'TwentySix'), True],
  [25, ('One', 'TwentySeven'), True],
  [26, ('One', 'TwentyEight'), True],
  [27, ('One', 'TwentyNine'), True],
  [28, ('One', 'Thirty'), True],
  [29, ('One', '

In [38]:
for j in p_2:
    print(j[0])

Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Two
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Three
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Four
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Five
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Six
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seven
Seve

In [50]:
%%time
tol = []
for i,j,k in zip(p_1, p_2, p):
    chi2 = chi_square(X=i[0], Y=j[0], Z=k, data=df, significance_level=0.05)
    tol.append((i[0],j[0],[x for x in k], chi2))
tol

CPU times: total: 2min 40s
Wall time: 2min 41s


[('One', 'Two', ['Three'], False),
 ('One', 'Two', ['Four'], True),
 ('One', 'Two', ['Five'], True),
 ('One', 'Two', ['Six'], True),
 ('One', 'Two', ['Seven'], False),
 ('One', 'Two', ['Eight'], False),
 ('One', 'Two', ['Nine'], False),
 ('One', 'Two', ['Ten'], True),
 ('One', 'Two', ['Eleven'], True),
 ('One', 'Two', ['Twelve'], False),
 ('One', 'Two', ['Thirteen'], False),
 ('One', 'Two', ['Fourteen'], False),
 ('One', 'Two', ['Fifteen'], False),
 ('One', 'Two', ['Sixteen'], True),
 ('One', 'Two', ['Seventeen'], False),
 ('One', 'Two', ['Eighteen'], False),
 ('One', 'Two', ['Nineteen'], False),
 ('One', 'Two', ['Twenty'], False),
 ('One', 'Two', ['TwentyOne'], False),
 ('One', 'Two', ['TwentyTwo'], False),
 ('One', 'Two', ['TwentyThree'], False),
 ('One', 'Two', ['TwentyFour'], False),
 ('One', 'Two', ['TwentyFive'], False),
 ('One', 'Two', ['TwentySix'], False),
 ('One', 'Two', ['TwentySeven'], False),
 ('One', 'Two', ['TwentyEight'], False),
 ('One', 'Two', ['TwentyNine'], False),


In [97]:
for i,j,k in zip(p_1, p_2, p):
    chi_square(X=i, Y=j, Z=[], data=df, significance_level=0.95)

KeyError: ('One',)

In [84]:
list_permutations

[('One', 'Two', 'Three', 'Four'),
 ('One', 'Two', 'Three', 'Five'),
 ('One', 'Two', 'Three', 'Six'),
 ('One', 'Two', 'Three', 'Seven'),
 ('One', 'Two', 'Three', 'Eight'),
 ('One', 'Two', 'Three', 'Nine'),
 ('One', 'Two', 'Three', 'Ten'),
 ('One', 'Two', 'Three', 'Eleven'),
 ('One', 'Two', 'Three', 'Twelve'),
 ('One', 'Two', 'Three', 'Thirteen'),
 ('One', 'Two', 'Three', 'Fourteen'),
 ('One', 'Two', 'Three', 'Fifteen'),
 ('One', 'Two', 'Three', 'Sixteen'),
 ('One', 'Two', 'Three', 'Seventeen'),
 ('One', 'Two', 'Three', 'Eighteen'),
 ('One', 'Two', 'Three', 'Nineteen'),
 ('One', 'Two', 'Three', 'Twenty'),
 ('One', 'Two', 'Three', 'TwentyOne'),
 ('One', 'Two', 'Three', 'TwentyTwo'),
 ('One', 'Two', 'Three', 'TwentyThree'),
 ('One', 'Two', 'Three', 'TwentyFour'),
 ('One', 'Two', 'Three', 'TwentyFive'),
 ('One', 'Two', 'Three', 'TwentySix'),
 ('One', 'Two', 'Three', 'TwentySeven'),
 ('One', 'Two', 'Three', 'TwentyEight'),
 ('One', 'Two', 'Three', 'TwentyNine'),
 ('One', 'Two', 'Three', 'Thi

In [80]:
%%time
p = [x for x in list_permutations if x[:2] not in remove_list]
len(p)

CPU times: total: 9.44 s
Wall time: 9.47 s


347480